In [ ]:
import torch
print(torch.__version__)  
print(torch.cuda.is_available())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(0)) 

In [ ]:
%pip install opencv-python-headless
%pip install opencv-python
%pip install scikit-image
%pip install scikit-learn
%pip install tqdm
%pip install sympy==1.13.3

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dir = 'D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Train/'


simple_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

temp_dataset = datasets.ImageFolder(root=train_dir, transform=simple_transform)
temp_loader = DataLoader(temp_dataset, batch_size=64, shuffle=False)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in temp_loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f"Mean: {mean}")
print(f"Std: {std}")


In [ ]:
mean_vals = mean 
std_vals = std

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.RandomApply([
        transforms.RandomRotation(degrees=5),
        transforms.ColorJitter(brightness=0.05, contrast=0.05),
        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
    ], p=0.6),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_vals, std=std_vals)
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_vals, std=std_vals)
])


In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cuda available: {torch.cuda.is_available()}")


train_dir = 'D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Train/'
test_dir = 'D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Test/'


train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_targets = torch.tensor(train_dataset.targets, dtype=torch.long)
class_sample_counts = torch.bincount(train_targets, minlength=len(train_dataset.classes))

weights_per_class = 1.0 / (class_sample_counts.float() + 1e-3)
sample_weights = weights_per_class[train_targets]

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(train_dataset),
    replacement=True
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    sampler=sampler,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

print(f"Train dataset size: {len(train_dataset)} images")
print(f"Test dataset size: {len(test_dataset)} images")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

original_path = "D:\Licenta\Datasets\ADNI_Oficial\Converted\Sagittal\AD\Sagittal_AD_018_S_4696_slice17.png"
processed_path = "D:\Licenta\Datasets\ADNI_Oficial\Processed\Sagittal\AD\Sagittal_AD_018_S_4696_slice17.png"

original_image = Image.open(original_path)
processed_image = Image.open(processed_path)


plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(original_image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(processed_image)
plt.title("Processed")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import torch


base_name = "Sagittal_AD_018_S_4696_slice17.png"
original_path = f"D:/Licenta/Datasets/ADNI_Oficial/Converted/Sagittal/AD/{base_name}"
processed_path = f"D:/Licenta/Datasets/ADNI_Oficial/Processed/Sagittal/AD/{base_name}"


original_image = Image.open(original_path)
processed_image = Image.open(processed_path)


train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(240),               
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(7),         
    transforms.ColorJitter(brightness=0.05, contrast=0.05), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


transformed_tensor = train_transform(processed_image)
transformed_image = transformed_tensor.permute(1, 2, 0).numpy()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transformed_image = transformed_image * std + mean
transformed_image = transformed_image.clip(0, 1)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(original_image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(processed_image)
plt.title("Processed")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(transformed_image)
plt.title("Augumented")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
sample, label = train_dataset[2]
print("Sample shape:", sample.shape)
print("Label:", label)


In [ ]:
import os

print("Train files:")
print(os.listdir('D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Train/'))

print("Test files:")
print(os.listdir('D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Test/'))


In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNetModel(nn.Module):
    def __init__(self, pretrained=True, num_classes=5):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)

        for name, param in self.resnet.named_parameters():
            if "layer3" in name or "layer4" in name or "fc" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.SiLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)

        targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).float().to(inputs.device)

        focal_weight = (1 - probs) ** self.gamma
        loss = -self.alpha * focal_weight * log_probs

        loss = (loss * targets_one_hot).sum(dim=1)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


In [ ]:
import torch
import matplotlib.pyplot as plt

class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']
train_targets = torch.tensor(train_dataset.targets, dtype=torch.long)
class_counts = torch.bincount(train_targets, minlength=len(class_names))
num_classes = len(class_names)
num_samples = len(train_targets)

weights_per_class = num_samples / (num_classes * class_counts.float())
class_weights = weights_per_class.to(device)

print("Class distribution and computed weights:")
for idx, (count, weight) in enumerate(zip(class_counts.tolist(), weights_per_class.tolist())):
    print(f"  Class {class_names[idx]} — samples: {count}, weight: {weight:.4f}")

plt.figure(figsize=(8, 6))
plt.bar(class_names, class_counts.cpu().numpy(), color='skyblue')
plt.xlabel('Class Label')
plt.ylabel('Number of Samples')
plt.title('Training Data Class Distribution')
plt.grid(axis='y')
plt.show()

criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import ReduceLROnPlateau

print(f"Using device: {device}")

model = ResNetModel(pretrained=True, num_classes=5).to(device)

optimizer = optim.AdamW(
    model.parameters(),
    lr=5e-5,                    
    betas=(0.9, 0.9999),
    weight_decay=2e-4
)


warmup_scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=5)
cosine_scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=15,            
    T_mult=2,
    eta_min=1e-6
)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)


scaler = GradScaler()


In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.amp import autocast, GradScaler
import time
from tqdm import tqdm
import copy
import os

def train_model(model, train_dataloader, optimizer, criterion, num_epochs, scaler, scheduler=None, patience=12, use_amp=True, run_name='experiment_focal'):
    writer = SummaryWriter(f'runs/{run_name}')

    train_losses = []
    train_accuracies = []

    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    total_start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        epoch_start_time = time.time()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for batch_idx, (images, labels) in enumerate(progress_bar):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type == 'cuda' and use_amp)):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.detach().item()
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

            current_loss = running_loss / (batch_idx + 1)
            current_acc = correct_preds / total_preds
            progress_bar.set_postfix(loss=current_loss, acc=current_acc)

        
        epoch_loss = running_loss / len(train_dataloader)
        epoch_acc = correct_preds / total_preds
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        if scheduler is not None:
            scheduler.step(epoch_loss)

        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Accuracy/train', epoch_acc, epoch)
        writer.add_scalar('LearningRate', optimizer.param_groups[0]['lr'], epoch)

        if epoch_loss < best_loss - 1e-4:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0

            os.makedirs('saved_models', exist_ok=True)
            torch.save(model.state_dict(), f'saved_models/best_model__sagittal_epoch{epoch+1}.pth')
            print(f" Saved new best model at epoch {epoch+1} with loss {best_loss:.4f}")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f" Early stopping triggered at epoch {epoch+1}")
            break

        epoch_end_time = time.time()
        print(f" Epoch {epoch+1}: Loss={epoch_loss:.4f}, Accuracy={epoch_acc*100:.2f}%")

    total_time = time.time() - total_start_time
    print(f"\n Total Training Time: {total_time/60:.2f} minutes")

    torch.save(model.state_dict(), 'saved_models/last_model_sagittal.pth')
    model.load_state_dict(best_model_wts)
    writer.close()

    return model, train_losses, train_accuracies


In [ ]:
from torch.amp import autocast
from sklearn.metrics import accuracy_score
import torch
from tqdm import tqdm

def evaluate_model(model, test_dataloader, criterion, device, writer=None, epoch=None, verbose=True, use_amp=True):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        progress_bar = tqdm(test_dataloader, desc="Evaluating", leave=False)

        for batch_idx, (images, labels) in enumerate(progress_bar):
            images, labels = images.to(device), labels.to(device)

            with autocast(device_type=device.type, dtype=torch.float16, enabled=(device.type == 'cuda' and use_amp)):
                outputs = model(images)
                loss = criterion(outputs, labels)

            
            loss_val = loss.item() if hasattr(loss, 'item') else float(loss)
            running_loss += loss_val

            _, predicted = torch.max(outputs, dim=1)
            all_preds.extend(predicted.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            current_loss = running_loss / (batch_idx + 1)
            current_acc = accuracy_score(all_labels, all_preds)
            progress_bar.set_postfix(loss=current_loss, acc=current_acc)

    epoch_loss = running_loss / len(test_dataloader)
    acc = accuracy_score(all_labels, all_preds)

    if verbose:
        print(f"\n Final Evaluation — Loss: {epoch_loss:.4f}, Accuracy: {acc * 100:.2f}%")

    if writer is not None and epoch is not None:
        writer.add_scalar('Loss/test', epoch_loss, epoch)
        writer.add_scalar('Accuracy/test', acc, epoch)

    return epoch_loss, acc, all_preds, all_labels


In [ ]:
%pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import torch

model, train_losses, train_accuracies = train_model(
    model,
    train_dataloader,
    optimizer,
    criterion,
    num_epochs=20,
    scaler=scaler,
    scheduler=scheduler,
    patience=12
)


epoch_loss, acc, all_preds, all_labels = evaluate_model(
    model,
    test_dataloader,
    criterion,
    device
)

plt.figure(figsize=(14, 6), dpi=100)

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss", color='red')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss per Epoch")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot([a * 100 for a in train_accuracies], label="Train Accuracy", color='green')
plt.axhline(acc * 100, color='blue', linestyle='--', linewidth=1.5, label='Test Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training Accuracy per Epoch")
plt.grid(True)
plt.legend()

plt.suptitle("Training Progress Overview", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("training_progress.png", bbox_inches='tight')
plt.show()


print(f"\nFinal Test Accuracy: {acc * 100:.2f}% | Final Test Loss: {epoch_loss:.4f}")

with open("final_test_score.txt", "w") as f:
    f.write(f"Final Test Accuracy: {acc * 100:.2f}%\n")
    f.write(f"Final Test Loss: {epoch_loss:.4f}\n")

file_path = 'ResNet_Alzheimer_Sagittal_Multiclass.pth'
torch.save(model.state_dict(), file_path)
print(f"Model saved at: {file_path}")


In [ ]:
model = ResNetModel(pretrained=False, num_classes=5)  
model.load_state_dict(torch.load('ResNet_Alzheimer_Sagittal_Multiclass.pth'))
model.to(device)
model.eval()


In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

image_path = "D:\Licenta\Datasets\ADNI_Oficial\Filtered\Sagittal\Test\CN\Sagittal_CN_002_S_4213_slice17.png"

if not os.path.isfile(image_path):
    print(" Image not found. Please check the path.")
    exit()


transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_vals, std=std_vals) 
])


class_names = ["AD", "CN", "EMCI", "LMCI", "MCI"]

def predict_image(model, image_path, device):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type=='cuda'):
        outputs = model(image_tensor)
        _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item(), outputs


def display_prediction(image_path, predicted_class, outputs):
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Predicted: {class_names[predicted_class]}")
    plt.show()

    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    print("\nScores per Class:")
    for i, prob in enumerate(probabilities[0]):
        print(f"  {class_names[i]:<5}: {prob.item():.4f}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

predicted_class, outputs = predict_image(model, image_path, device)
display_prediction(image_path, predicted_class, outputs)


In [ ]:
import random
import matplotlib.pyplot as plt
import torch
import numpy as np

class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']

def show_random_predictions(model, test_dataloader, num_images=12, cols=4, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.eval()
    all_images = []
    all_labels = []

    for images, labels in test_dataloader:
        all_images.append(images)
        all_labels.append(labels)

    if not all_images:
        print(" Test dataloader appears to be empty. Make sure it's not already exhausted.")
        return

    all_images = torch.cat(all_images, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    total_images = len(all_images)
    num_images = min(num_images, total_images)
    indices = random.sample(range(total_images), num_images)

    correct_list = []
    wrong_list = []

    for idx in indices:
        image = all_images[idx]
        label = all_labels[idx].item()

       
        input_tensor = image.unsqueeze(0).to(device)

        with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
            outputs = model(input_tensor)
            _, predicted_class = torch.max(outputs, 1)

        pred_idx = predicted_class.item()

       
        img_np = image.cpu().detach().numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img_np = img_np * std + mean
        img_np = np.clip(img_np, 0, 1)

        entry = {
            'image': img_np,
            'pred': class_names[pred_idx],
            'true': class_names[label],
            'is_correct': pred_idx == label
        }

        if entry['is_correct']:
            correct_list.append(entry)
        else:
            wrong_list.append(entry)

    all_entries = wrong_list + correct_list  
    total = len(all_entries)
    rows = total // cols + int(total % cols != 0)

    plt.figure(figsize=(cols * 3.5, rows * 3.5))

    for i, entry in enumerate(all_entries):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(entry['image'])
        title_color = 'green' if entry['is_correct'] else 'red'
        plt.title(f"Pred: {entry['pred']}\nTrue: {entry['true']}", color=title_color)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nSummary:")
    print(f"Total images shown: {total}")
    print(f"Correct predictions: {len(correct_list)}")
    print(f"Wrong predictions:  {len(wrong_list)}")

    if wrong_list:
        print("\n🔍 Wrong predictions detail:")
        for idx, entry in enumerate(wrong_list, start=1):
            print(f"  {idx}. Predicted: {entry['pred']} | Actual: {entry['true']}")


In [ ]:
show_random_predictions(model, test_dataloader, num_images=100, cols=6, device=device)


In [ ]:
%pip install seaborn

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import os
from datetime import datetime

class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_preds = []
all_labels = []

model = model.to(device)
model.eval()

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


cm = confusion_matrix(all_labels, all_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)


plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, square=True)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Absolute Numbers)')
plt.tight_layout()
plt.savefig(f"{save_dir}/confusion_matrix_absolute_{timestamp}.png")
plt.show()


plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names, square=True)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Normalized - Percentages)')
plt.tight_layout()
plt.savefig(f"{save_dir}/confusion_matrix_normalized_{timestamp}.png")
plt.show()


print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))


In [ ]:
import os
import random
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from datetime import datetime

class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']

predict_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def predict_random_image_from_folder(model, folder_path, device, save_fig=False):
    images_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not images_list:
        print("No images found in the selected folder.")
        return

    image_path = random.choice(images_list)
    print(f"\nSelected image: {image_path}")

    image = Image.open(image_path).convert('RGB')
    input_tensor = predict_transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
        outputs = model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        top_probs, top_indices = torch.topk(probabilities, 3)

  
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Predicted: {class_names[top_indices[0][0].item()]}", fontsize=14)
    plt.tight_layout()

    if save_fig:
        os.makedirs("random_predictions", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"random_predictions/pred_{timestamp}.png"
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved prediction image to: {save_path}")

    plt.show()

    print("\n Top 3 Predictions:")
    for i in range(3):
        idx = top_indices[0][i].item()
        prob = top_probs[0][i].item()
        print(f"{class_names[idx]:<5}: {prob:.4f}")


In [ ]:
%pip install grad-cam


In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import os
from datetime import datetime

class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']

gradcam_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def generate_gradcam(model, image_path, device, save=False):
    from pytorch_grad_cam import GradCAM
    from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
    from pytorch_grad_cam.utils.image import show_cam_on_image

    model.eval()

   
    image = Image.open(image_path).convert('RGB')
    input_tensor = gradcam_transform(image).unsqueeze(0).to(device)
    image_np = np.array(image.resize((256, 256))).astype(np.float32) / 255.0

    target_layers = [model.resnet.layer4[-1]]

    with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
        outputs = model(input_tensor)
        _, pred_class = torch.max(outputs, 1)

    targets = [ClassifierOutputTarget(pred_class.item())]

    
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  

    cam_image = show_cam_on_image(image_np, grayscale_cam, use_rgb=True)

    plt.figure(figsize=(6, 6))
    plt.imshow(cam_image)
    plt.axis('off')
    plt.title(f"Grad-CAM - Predicted: {class_names[pred_class.item()]}")
    plt.tight_layout()

    if save:
        os.makedirs("gradcam_outputs", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"gradcam_outputs/gradcam_{class_names[pred_class.item()]}_{timestamp}.png"
        plt.savefig(save_path, bbox_inches='tight')
        print(f" Grad-CAM saved to: {save_path}")

    plt.show()



In [ ]:
generate_gradcam(model, "D:\Licenta\Datasets\ADNI_Oficial\Filtered\Sagittal\Test\AD\Sagittal_AD_019_S_4549_slice16.png", device, save=True)

In [ ]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix

def top3_best_classes(model, test_dataloader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)

            with torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
                outputs = model(images)
                _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(class_names))))

    class_totals = cm.sum(axis=1)
    class_accuracy = np.zeros(len(class_names))

    for i in range(len(class_names)):
        if class_totals[i] > 0:
            class_accuracy[i] = cm[i, i] / class_totals[i]
        else:
            class_accuracy[i] = np.nan

    sorted_indices = np.argsort(class_accuracy)[::-1]

    print("\n Top 3 Best Predicted Classes:")
    for idx in sorted_indices[:3]:
        acc = class_accuracy[idx]
        if not np.isnan(acc):
            print(f"  {class_names[idx]} — Accuracy: {acc:.4f}")
        else:
            print(f"  {class_names[idx]} — No samples in test set")

    print("\nFull Class Accuracy:")
    for idx in sorted_indices:
        acc = class_accuracy[idx]
        if not np.isnan(acc):
            print(f"  {class_names[idx]:<5} — Accuracy: {acc:.4f}")
        else:
            print(f"  {class_names[idx]:<5} —  No samples")


In [ ]:
top3_best_classes(model, test_dataloader, device, class_names)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def top_classes_sorted(model, test_dataloader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)

            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)
                _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)

    class_accuracy = cm.diagonal() / cm.sum(axis=1)

    sorted_indices_desc = np.argsort(class_accuracy)[::-1]
    
    print("\nTop Classes (Best to Worst):")
    for idx in sorted_indices_desc:
        print(f"{class_names[idx]} — Accuracy: {class_accuracy[idx]:.4f}")

    sorted_indices_asc = np.argsort(class_accuracy)

    print("\nTop Classes (Worst to Best):")
    for idx in sorted_indices_asc:
        print(f"{class_names[idx]} — Accuracy: {class_accuracy[idx]:.4f}")


In [ ]:
top_classes_sorted(model, test_dataloader, device, class_names)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_auc(model, test_dataloader, device, class_names):
    model.eval()
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)

            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)

            all_outputs.append(outputs.cpu())
            all_labels.append(labels.cpu())

    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)

    
    all_labels_binarized = label_binarize(all_labels, classes=list(range(len(class_names))))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(all_labels_binarized[:, i], all_outputs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

   
    plt.figure(figsize=(10, 8))
    for i in range(len(class_names)):
        plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC = {roc_auc[i]:.2f})")

    plt.plot([0, 1], [0, 1], 'k--')  
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (One vs Rest)')
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()


In [ ]:
plot_roc_auc(model, test_dataloader, device, class_names)

In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNet101_MRI(nn.Module):
    def __init__(self, pretrained=True, num_classes=5):
        super(ResNet101_MRI, self).__init__()
        self.resnet = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1 if pretrained else None)

        for name, param in self.resnet.named_parameters():
            if "layer2" in name or "layer3" in name or "layer4" in name or "fc" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.amp import GradScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_r101 = ResNet101_MRI(pretrained=True, num_classes=5).to(device)

optimizer_r101 = optim.AdamW(
    model_r101.parameters(),
    lr=5e-5,
    weight_decay=1e-4
)


scheduler_r101 = ReduceLROnPlateau(
    optimizer_r101,
    mode='min',
    factor=0.5,
    patience=4,
)


scaler_r101 = GradScaler()


model_r101, train_losses_r101, train_acc_r101 = train_model(
    model_r101,
    train_dataloader,
    optimizer_r101,
    criterion,
    num_epochs=20,
    scaler=scaler_r101,
    scheduler=scheduler_r101,  
    patience=12
)


In [ ]:
epoch_loss_r101, acc_r101, preds_r101, labels_r101 = evaluate_model(
    model_r101,
    test_dataloader,
    criterion,
    device
)

print(f"\nModel 2  - Final Test Accuracy: {acc_r101*100:.2f}% | Final Test Loss: {epoch_loss_r101:.4f}")

In [ ]:
file_path = 'ResNet101_Alzheimer_Sagittal_Multiclass.pth'
torch.save(model_r101.state_dict(), file_path)
print(f" ResNet101 model saved at: {file_path}")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def ensemble_resnet50_resnet101(model_r50, model_r101, dataloader, device, class_names, w_r50=0.5, w_r101=0.5, save_report=False):
    assert abs(w_r50 + w_r101 - 1.0) < 1e-5, " Weights must sum to 1.0"

    model_r50.eval()
    model_r101.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            with torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
                out_r50 = model_r50(images)
                out_r101 = model_r101(images)

            combined_logits = w_r50 * out_r50 + w_r101 * out_r101
            _, predicted = torch.max(combined_logits, dim=1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)

    print(f"\nEnsemble (ResNet50 + ResNet101) Accuracy: {acc * 100:.2f}%")
    print("\nClassification Report:\n")
    print(report)

    if save_report:
        with open("ensemble_report.txt", "w") as f:
            f.write(f"Ensemble Accuracy: {acc * 100:.2f}%\n\n")
            f.write(report)

    return acc, all_preds, all_labels


In [ ]:
class_names = ['AD', 'CN', 'EMCI', 'LMCI', 'MCI']
ensemble_acc, ensemble_preds, ensemble_labels = ensemble_resnet50_resnet101(
    model_r50=model,
    model_r101=model_r101,
    dataloader=test_dataloader,
    device=device,
    class_names=class_names,
    w_r50=0.4,
    w_r101=0.6,
    save_report=True
)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(ensemble_labels, ensemble_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Ensemble Confusion Matrix")
plt.show()


In [ ]:
best_acc = 0.0
best_weights = (0.5, 0.5)

for w_r50 in [0.1 * i for i in range(1, 10)]:
    w_r101 = 1.0 - w_r50

    acc_ens, _, _ = ensemble_resnet50_resnet101(
        model_r50=model,
        model_r101=model_r101,
        dataloader=test_dataloader,
        device=device,
        class_names=class_names,
        w_r50=w_r50,
        w_r101=w_r101,
        save_report=False  
    )

    print(f" w_r50 = {w_r50:.1f} | w_r101 = {w_r101:.1f} → Accuracy: {acc_ens * 100:.2f}%")

    if acc_ens > best_acc:
        best_acc = acc_ens
        best_weights = (w_r50, w_r101)

print(f"\n Best ensemble accuracy: {best_acc * 100:.2f}% with weights → ResNet50: {best_weights[0]:.1f}, ResNet101: {best_weights[1]:.1f}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

epoch_loss1, acc1, preds1, labels1 = evaluate_model(
    model, test_dataloader, criterion, device
)


epoch_loss2, acc2, preds2, labels2 = evaluate_model(
    model_r101, test_dataloader, criterion, device
)


acc_ens, preds_ens, labels_ens = ensemble_resnet50_resnet101(
    model_r50=model,
    model_r101=model_r101,
    dataloader=test_dataloader,
    device=device,
    class_names=class_names,
    w_r50=0.5,
    w_r101=0.5
)

models_names = ["ResNet50", "ResNet101", "Ensemble"]
accuracies = [acc1 * 100, acc2 * 100, acc_ens * 100]

plt.figure(figsize=(8, 5))
bars = plt.bar(models_names, accuracies, color=["red", "blue", "green"])
plt.ylim(0, 100)
plt.ylabel("Accuracy (%)")
plt.title("Model Comparison on Test Set")

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1,
             f"{height:.2f}%", ha='center', va='bottom', fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
def generate_gradcam_for_wrong_predictions(model, dataloader, device, class_names, model_type="resnet", save_dir="gradcam_wrong_predictions"):
    model.eval()
    os.makedirs(save_dir, exist_ok=True)

    transform_to_rgb = transforms.ToPILImage()

    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)

        with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

        for i in range(len(images)):
            if preds[i] != labels[i]:
                image_tensor = images[i].unsqueeze(0)
                input_tensor = image_tensor.to(device)

                rgb_image = transform_to_rgb(images[i].cpu())

                resize_dim = (256, 256) if model_type == "resnet" else (384, 384)
                image_np = np.array(rgb_image.resize(resize_dim)).astype(np.float32) / 255.0

                if model_type == "resnet":
                    target_layers = [model.resnet.layer4[-1]]
                elif model_type == "efficientnet":
                    target_layers = [model.model.features[-1]]
                else:
                    raise ValueError("Unsupported model_type. Use 'resnet' or 'efficientnet'.")

                cam = GradCAM(model=model, target_layers=target_layers)
                grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(preds[i].item())])[0]
                cam_image = show_cam_on_image(image_np, grayscale_cam, use_rgb=True)

                true_label = class_names[labels[i].item()]
                pred_label = class_names[preds[i].item()]
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                filename = f"true_{true_label}_pred_{pred_label}_{timestamp}.png"
                save_path = os.path.join(save_dir, filename)

                plt.imsave(save_path, cam_image)
                print(f" Saved: {save_path}")


In [ ]:

generate_gradcam_for_wrong_predictions(model, test_dataloader, device, class_names, model_type="resnet")


In [ ]:
def show_random_ensemble_predictions(model_r50, model_r101, test_dataloader, class_names, device=None, w_r50=0.5, w_r101=0.5, num_images=12, cols=6):
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import random

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_r50.eval()
    model_r101.eval()
    all_images = []
    all_labels = []

    for images, labels in test_dataloader:
        all_images.append(images)
        all_labels.append(labels)

    all_images = torch.cat(all_images, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    total_images = len(all_images)
    num_images = min(num_images, total_images)
    indices = random.sample(range(total_images), num_images)

    correct_list = []
    wrong_list = []

    for idx in indices:
        image = all_images[idx]
        label = all_labels[idx].item()
        input_tensor = image.unsqueeze(0).to(device)

        with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=device.type == 'cuda'):
            out_r50 = model_r50(input_tensor)
            out_r101 = model_r101(input_tensor)
            combined = w_r50 * out_r50 + w_r101 * out_r101
            _, predicted_class = torch.max(combined, 1)

        pred_idx = predicted_class.item()

        img_np = image.cpu().detach().numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img_np = img_np * std + mean
        img_np = np.clip(img_np, 0, 1)

        entry = {
            'image': img_np,
            'pred': class_names[pred_idx],
            'true': class_names[label],
            'is_correct': pred_idx == label
        }

        if entry['is_correct']:
            correct_list.append(entry)
        else:
            wrong_list.append(entry)

    all_entries = wrong_list + correct_list
    rows = len(all_entries) // cols + int(len(all_entries) % cols != 0)

    plt.figure(figsize=(cols * 3.5, rows * 3.5))

    for i, entry in enumerate(all_entries):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(entry['image'])
        title_color = 'green' if entry['is_correct'] else 'red'
        plt.title(f"Pred: {entry['pred']}\nTrue: {entry['true']}", color=title_color)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nSummary:")
    print(f"Total images shown: {len(all_entries)}")
    print(f"Correct predictions: {len(correct_list)}")
    print(f"Wrong predictions:  {len(wrong_list)}")

    if wrong_list:
        print("\n🔍 Wrong predictions detail:")
        for i, entry in enumerate(wrong_list, 1):
            print(f"  {i}. Predicted: {entry['pred']} | Actual: {entry['true']}")


In [ ]:
show_random_ensemble_predictions(
    model_r50=model,
    model_r101=model_r101,
    test_dataloader=test_dataloader,
    class_names=class_names,
    device=device,
    w_r50=0.45,
    w_r101=0.55,
    num_images=100,
    cols=6
)


In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

image_path = "D:\Licenta\Datasets\ADNI_Oficial\Filtered\Sagittal\Test\CN\Sagittal_CN_002_S_1261_slice24.png"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.isfile(image_path):
    print("Image not found. Please check the path.")
    exit()


transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_vals, std=std_vals) 
])


class_names = ["AD", "CN", "EMCI", "LMCI", "MCI"]

def predict_ensemble(model_r50, model_r101, image_path, device, w_r50=0.5, w_r101=0.5):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    model_r50.eval()
    model_r101.eval()

    with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
        out_r50 = model_r50(image_tensor)
        out_r101 = model_r101(image_tensor)
        ensemble_output = w_r50 * out_r50 + w_r101 * out_r101
        _, predicted_class = torch.max(ensemble_output, dim=1)

    return predicted_class.item(), ensemble_output

def display_prediction(image_path, predicted_class, outputs):
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Predicted (Ensemble): {class_names[predicted_class]}")
    plt.show()

    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    print("\nScores per Class:")
    for i, prob in enumerate(probabilities[0]):
        print(f"  {class_names[i]:<5}: {prob.item():.4f}")

model_r50 = model.to(device)
model_r101 = model_r101.to(device)

predicted_class, outputs = predict_ensemble(model_r50, model_r101, image_path, device)
display_prediction(image_path, predicted_class, outputs)


In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F

image_path = "D:\Licenta\Datasets\ADNI_Oficial\Filtered\Sagittal\Test\AD\Sagittal_AD_019_S_4549_slice16.png"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.isfile(image_path):
    print("Image not found. Please check the path.")
    exit()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_vals, std=std_vals)  
])


class_names = ["AD", "CN", "EMCI", "LMCI", "MCI"]


def predict_model(model, image_tensor, device):
    model.eval()
    with torch.no_grad(), torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
        outputs = model(image_tensor)
        probs = F.softmax(outputs, dim=1)
        top2 = torch.topk(probs, 2)
        return outputs, probs, top2


image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)


model_r50 = model_r50.to(device)
model_r101 = model_r101.to(device)


out50, probs50, top2_50 = predict_model(model_r50, image_tensor, device)
out101, probs101, top2_101 = predict_model(model_r101, image_tensor, device)

ensemble_logits = 0.5 * out50 + 0.5 * out101
probs_ens = F.softmax(ensemble_logits, dim=1)
top2_ens = torch.topk(probs_ens, 2)

plt.imshow(image)
plt.axis('off')
plt.title("MRI Image Prediction")
plt.show()

def print_top2(name, top2, probs):
    print(f"\n {name} Prediction:")
    for i in range(2):
        idx = top2.indices[0][i].item()
        conf = probs[0][idx].item()
        print(f"  {i+1}. {class_names[idx]} — {conf:.4f}")

print_top2("ResNet50", top2_50, probs50)
print_top2("ResNet101", top2_101, probs101)
print_top2("Ensemble", top2_ens, probs_ens)
